## 2024 Interactive Predictions

In [12]:
from __future__ import print_function
import pickle
#import time
import sys
sys.path.append('..')
import swagger_client as v3client
from swagger_client.rest import ApiException
from featurization import addMatch, featurizeMatch, invertMatch, featurizeAlliances
import ipywidgets as widgets

from scipy.sparse import csr_array
from scipy.sparse.linalg import spsolve, norm
import numpy as np
from scipy.sparse.linalg import lsqr
from scipy.linalg import lstsq

filename = 'matches_2024.pkl'
matches = []
with open(filename, 'rb') as f:
    matches = pickle.load(f)

pnw_district = [m.key for m in matches['events'] ]
    
print(f'{len(matches["matches"])} events')

non_empty = [k for k in matches['matches'].keys() if len(matches['matches'][k])>0]
data = [m for k in matches['matches'] for m in matches['matches'][k]]
data = [m for m in data if m.winning_alliance!='' and m.score_breakdown is not None]
print(f'Found {len(data)} matches')


pnw_teams = set()
for m in [m for m in data if m.event_key in pnw_district]:
    for t in m.alliances.red.team_keys:
        pnw_teams.add(t)
    for t in m.alliances.blue.team_keys:
        pnw_teams.add(t)
    
pnw_teams = list(sorted(pnw_teams))
print(f'PNW Teams: {len(pnw_teams)}')

pnw_matches = [m for m in data] # if m.event_key in pnw_district]
pnw_lookup = dict((k,i) for (i,k) in enumerate(pnw_teams))

qualifiers = [x for x in data if x.comp_level=='qm'] 

def get_rows(m):
    yield m.alliances.red.team_keys, m.score_breakdown['red']['totalPoints']
    yield m.alliances.blue.team_keys, m.score_breakdown['blue']['totalPoints']


A_data = []
row = []
col = []
b = []
ctr = 0
for m in map(get_rows, pnw_matches):
    for r in m:
        for t in r[0]:
            row.append(len(b))
            col.append(pnw_lookup[t])
            A_data.append(1)
        b.append(r[1])
b = np.array(b)            
A = csr_array((A_data, (row, col)), shape=(len(b), len(pnw_lookup)))
print(A.shape, b.shape )
#x = spsolve(A, b)
#x

# Thanks ChatGPT!
x, residuals, rank, s = lstsq(A.todense(), b)

RSS = residuals.sum()
Rinv = np.linalg.inv(np.triu(s))

sigmas = np.sqrt(RSS / (len(b) - len(x)) * np.diag(Rinv))

#return_values = lsqr(A, b, calc_var=True)
#result = return_values
#print(result)
#x = return_values[0]
#var = return_values[-1]
#print(var)

opr = [(t,x[i],sigmas[i]) for i,t in enumerate(pnw_teams)]
#for t,opr,sigma in sorted(opr, key=lambda x: x[1], reverse=True):
#    print(t,opr,sigma)

#print((A@x).shape,b.shape)
#print(A@x-b)
err = np.mean(A@x-b)
print(f'Error: {err}')
opr_lookup = dict([(t,(x[i],sigmas[i])) for i,t in enumerate(pnw_teams)])
opr_lookup[''] = (0,0)

def predict(red,blue):
    mu = []
    sigma = []
    for r in red:
        mu.append(opr_lookup[r][0])
        sigma.append(opr_lookup[r][1])
    for b in blue:
        mu.append(-opr_lookup[b][0])
        sigma.append(opr_lookup[b][1])
    mu = sum(mu)
    sigma = np.linalg.norm(sigma)
    return(mu,sigma)

def get_event_teams(event):
    event_teams = set()
    for m in [m for m in data if m.event_key == selected]:
        for t in m.alliances.red.team_keys:
            event_teams.add(t)
        for t in m.alliances.blue.team_keys:
            event_teams.add(t)
    return list(sorted(event_teams))

184 events
Found 4580 matches
PNW Teams: 2139
(9160, 2139) (9160,)
Error: 2.3208994163692356e-14


In [24]:
match_names = list([m for m in matches['matches'].keys() if len(matches['matches'][m])>0])
dropdown = widgets.Dropdown(
    options=match_names,
    value=None,
    description='Event:',
    disabled=False,
)
display(dropdown)

caption = widgets.Label(value='Selected Match: [None]')
selected = None
event_teams = None
alliance_widgets = None
def handle_match_change(change):
    global selected
    global event_teams
    global alliance_widgets
    selected = change.new
    event_teams = get_event_teams(selected)
    event_matches = matches['matches'][selected]
    caption.value = f'Selected Event: {selected} ({len(event_matches)} matches, {len(event_teams)} event teams)'
    alliance_widgets = [widgets.Dropdown(options=['', *event_teams]) for _ in range(24) ]
    opr_event = filter(lambda x: x[0] in event_teams, opr)
    for i, (t,opr_value,sigma) in enumerate(sorted(opr_event, key=lambda x: x[1], reverse=True)):
        print(t,opr_value,sigma)
    display(widgets.GridBox(alliance_widgets, layout=widgets.Layout(grid_template_columns="repeat(3, 100px)")))

dropdown.observe(handle_match_change, names='value')

display(caption)

    

Dropdown(description='Event:', options=('2024arli', '2024bcvi', '2024brbr', '2024caph', '2024casf', '2024casj'…

Label(value='Selected Match: [None]')

frc2910 38.73062004495877 6.163535165975137
frc360 31.166471533424524 6.511281064373579
frc2522 28.153561108825023 5.970250743930992
frc1778 28.12675813005383 5.5667017749053525
frc4682 24.51118910645359 6.790013766854034
frc4450 23.8629701675581 6.722815446630101
frc948 23.089148173714467 9.240837033417328
frc4089 21.60371188945792 6.630068055331495
frc2980 20.681307581675874 6.216961527488701
frc5941 20.11363846319001 7.108069829993734
frc3681 19.94140683705818 6.542627914832828
frc4180 19.25483692369 6.649944662975433
frc2930 19.118842078614584 6.176061767473734
frc9036 18.880243925281967 8.243157417531297
frc4513 17.215886045235248 6.7440396395698645
frc5827 15.442466594166943 7.063071250816447
frc4131 13.637300639185424 6.64396611367873
frc2412 13.372339630615373 5.90563623134834
frc4915 13.203208951084104 6.83273917162271
frc1318 13.095002160190823 5.18340432612387
frc7627 12.418576291581763 7.55543919201937
frc3049 11.969977561513366 6.268985803235005
frc9450 11.952302543459234 

GridBox(children=(Dropdown(options=('', 'frc1318', 'frc1778', 'frc2412', 'frc2522', 'frc2903', 'frc2910', 'frc…

In [26]:
from collections import Counter
from tqdm import tqdm

alliances =  dict([(f'A{i+1}', [alliance_widgets[3*i+k].value for k in range(3)]) for i in range(8)])



bracket = {
    1: ['A1', 'A8'],
    2: ['A4', 'A5'],
    3: ['A2', 'A7'],
    4: ['A3', 'A6'],
    5: ['L1', 'L2'],
    6: ['L3', 'L4'],
    7: ['W1', 'W2'],
    8: ['W3', 'W4'],
    9: ['L7', 'W6'],
    10: ['W5', 'L8'],
    11: ['W7', 'W8'],
    12: ['W10', 'W9'],
    13: ['L11', 'W12'],
    14: ['W11', 'W13'],
    15: ['W14', 'L14'],
    16: ['W15', 'L15']
}

density = {i:Counter() for i in range(1,len(bracket)+1)}
        
def runMatch(matchNumber):
    red_id,blue_id = bracket[matchNumber]
    
    red = alliances[red_id]
    blue =alliances[blue_id]
    density[matchNumber][str(red)]+=1
    density[matchNumber][str(blue)]+=1
    #density[matchNumber][red_id]+=1
    #density[matchNumber][blue_id]+=1
    
    # mu and sigma are the expected advantage for red
    mu,sigma = predict(red,blue)
    r = np.random.normal(mu,sigma)
    #print(red,blue,mu,sigma,r)
    
    if r>0:        
        winner = red
        loser = blue
    else:
        winner = blue
        loser = red
    alliances[f'W{matchNumber}'] = winner
    alliances[f'L{matchNumber}'] = loser
    #print(f'{winner} beats {loser} by {abs(r)} in match {matchNumber}')

    
def pMatch(matchNumber):
    red_id,blue_id = bracket[matchNumber]
    
    red = alliances[red_id]
    blue =alliances[blue_id]
    density[matchNumber][str(red)]+=1
    density[matchNumber][str(blue)]+=1
    #density[matchNumber][red_id]+=1
    #density[matchNumber][blue_id]+=1
    
    # mu and sigma are the expected advantage for red
    return predict(red,blue)

import scipy.stats as stats

def pRed(matchNumber):
    mu,sigma = pMatch(matchNumber)
    return 1.0-stats.norm.cdf(0, loc=mu, scale=sigma)
    
    
def runBracket():
    for i in range(1,17):
        runMatch(i)        
    wins = Counter()
    for i in range(14,17):
        w = alliances[f'W{i}']
        wins[str(w)]+=1
    return sorted(wins, reverse=True, key=lambda x: wins[x])[0], (alliances['A6'] in [alliances['W11'],alliances['W13']])

overall = Counter()
inFinalCtr = 0
for b in tqdm(range(1000)):
    (w, inFinal) = runBracket()
    overall[w] += 1
    inFinalCtr += 1 if inFinal else 0
        
for k in sorted(overall, key=lambda x: overall[x], reverse=True):
    print(k, overall[k])

print(f'inFinal: {inFinalCtr}')

for k in sorted(density):
    print(k, density[k])

100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 2603.83it/s]

['frc2910', 'frc3588', 'frc360'] 882
['frc2522', 'frc2903', 'frc1778'] 118
inFinal: 0
1 Counter({"['frc2910', 'frc3588', 'frc360']": 1000, "['', '', '']": 1000})
2 Counter({"['', '', '']": 2000})
3 Counter({"['frc2522', 'frc2903', 'frc1778']": 1000, "['', '', '']": 1000})
4 Counter({"['frc3268', 'frc4682', 'frc949']": 1000, "['', '', '']": 1000})
5 Counter({"['', '', '']": 2000})
6 Counter({"['', '', '']": 1989, "['frc3268', 'frc4682', 'frc949']": 11})
7 Counter({"['frc2910', 'frc3588', 'frc360']": 1000, "['', '', '']": 1000})
8 Counter({"['frc2522', 'frc2903', 'frc1778']": 1000, "['frc3268', 'frc4682', 'frc949']": 989, "['', '', '']": 11})
9 Counter({"['', '', '']": 1989, "['frc3268', 'frc4682', 'frc949']": 11})
10 Counter({"['', '', '']": 1011, "['frc3268', 'frc4682', 'frc949']": 938, "['frc2522', 'frc2903', 'frc1778']": 51})
11 Counter({"['frc2910', 'frc3588', 'frc360']": 1000, "['frc2522', 'frc2903', 'frc1778']": 949, "['frc3268', 'frc4682', 'frc949']": 51})
12 Counter({"['', '', '